---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [9]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple

In [10]:
@dataclass(frozen=True)
class Node:
    """Basic immutable graph node."""

    name: str

    def __str__(self) -> str:
        return self.name

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the sources and destinations, but bidirectional edge cannot have them. Which node should be returned by a bidirectional edge's "source" property? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional. Together these two facts mean neither can inherit from the other.

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we can build undirected graph with directed edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird.

BaseEdge is a abstract base class for edges. An edge has two nodes, node1 and node2, can return them in a tuple (.nodes) and can be represented by a string where names of both nodes are separated by a separator. Also is hashable, which means it implements \_\_eq\_\_ and \_\_hash\_\_ methods.
_separator, \_\_eq\_\_ and \_\_hash\_\_ are abstract methods. Their implementation depends on whether the edge is bidirectional or unidirectional.

@ property means accessing `.nodes` on in instance will call this *method*, which works as a getter. It also blocks setting the attribute (via `.nodes = something`). But it doesn't block its mutation, were it e.g. a list.

In [ ]:
@dataclass(frozen=True)
class BaseEdge(ABC):
    """Abstract base class for graph edge."""

    node1: Node
    node2: Node
    weight: int | float | None = None

    @property
    def weighted(self) -> bool:
        return self.weight is not None

    @property
    def nodes(self) -> Tuple[Node, Node]:
        return self.node1, self.node2

    @property
    @abstractmethod
    def _separator(self) -> str:
        pass

    def __str__(self) -> str:
        node1, node2 = self.nodes
        return f"{node1} {self._separator} {node2}"


@dataclass(frozen=True)
class Edge(BaseEdge):
    """Bidirectional graph edge."""

    def __init__(self, node1: Node, node2: Node, weight: int | float | None = None):
        # Canonical ordering for dataclass's __eq__ and __hash__
        # to work correctly and for better str representation
        if node1.name > node2.name:
            BaseEdge.__init__(self, node2, node1, weight)
        else:
            BaseEdge.__init__(self, node1, node2, weight)

    @property
    def _separator(self) -> str:
        if self.weighted:
            return f"<--({self.weight})-->"
        else:
            return "<---->"


@dataclass(frozen=True)
class DirectedEdge(BaseEdge):
    """Unidirectional graph edge."""

    def __init__(self, src: Node, dest: Node, weight: int | float | None = None):
        # Improves signature of the constructor
        # Invariant: src=node1 and dest=node2
        BaseEdge.__init__(self, src, dest, weight)

    @property
    def src(self) -> Node:
        return self.nodes[0]

    @property
    def dest(self) -> Node:
        return self.nodes[1]

    @property
    def _separator(self) -> str:
        if self.weighted:
            return f"---({self.weight})-->"
        else:
            return "----->"

### Testing of Nodes and Edges

In [12]:
node1 = Node("1")
node2 = Node("2")

# BaseEdge()  # TypeError: Can't instantiate abstract class BaseEdge without an implementation for abstract method '_separator'

e12 = Edge(node1, node2)
we12_314 = Edge(node1, node2, 3.14)
dir12 = DirectedEdge(node1, node2)
wdir12_314 = DirectedEdge(node1, node2, 3.14)

e21 = Edge(node2, node1)
we21_314 = Edge(node2, node1, 3.14)
dir21 = DirectedEdge(node2, node1)
wdir21_314 = DirectedEdge(node2, node1, 3.14)

we12_3 = Edge(node1, node2, 3)
wdir12_3 = DirectedEdge(node1, node2, 3)

In [13]:
print(f"{e12 == e21}: {e12} equals {e21}")
print(f"{we12_314 == we21_314}: {we12_314} equals {we21_314}")
print(f"{dir12 == dir21}: {dir12} equals {dir21}")
print(f"{wdir12_314 == wdir21_314}: {wdir12_314} equals {wdir21_314}")
print(f"{we12_314 == we12_3}: {we12_314} equals {we12_3}")
print(f"{wdir12_314 == wdir12_3}: {wdir12_314} equals {wdir12_3}")

True: 1 <----> 2 equals 1 <----> 2
True: 1 <--(3.14)--> 2 equals 1 <--(3.14)--> 2
False: 1 -----> 2 equals 2 -----> 1
False: 1 ---(3.14)--> 2 equals 2 ---(3.14)--> 1
False: 1 <--(3.14)--> 2 equals 1 <--(3)--> 2
False: 1 ---(3.14)--> 2 equals 1 ---(3)--> 2


In [14]:
print({e12, e21})  # exactly what we wanted because they are the same edge!
print({we12_314, we21_314})
print({node1, Node("1"), node2})

{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=None)}
{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=3.14)}
{Node(name='2'), Node(name='1')}


In [15]:
class C:
    pass


{C(), C()}

{<__main__.C at 0x2bc7844ad50>, <__main__.C at 0x2bc78758830>}

In [ ]:
class Digraph:
    """Directed graph."""

    # nodes is a list of the nodes in the graph.
    # edges is a dict mapping each node to a list of its children.
    # Implementation of these children depends on weighted: if True, children
    # implemented as tuples, where the first element is the child node
    # and the second element is the weight of the edge. When False, the only
    # element is the node. The first added edge determines the Digraph
    # for the rest of its life.
    # Internally, an "edge" is one-way only.
    # Bidirectional edges treated as having an "edge" in both directions.

    def __init__(self, weighted=None):
        self._nodes = []
        self._edges = {}
        self.weighted = weighted

    def add_node(self, node):
        if self.has_node(node):
            raise ValueError("Duplicate node")
        else:
            self._nodes.append(node)
            self._edges[node] = []

    def _check_weighted(self, edge):
        if edge.weighted and not self.weighted:
            raise ValueError("This graph is unweighted, cannot add weighted edge")
        if self.weighted and not edge.weighted:
            raise ValueError("This graph is weighted, cannot add unweighted edge")

    def add_directed_edge(self, edge):
        nodes = edge.nodes
        if not (self.has_node(nodes[0]) and self.has_node(nodes[1])):
            raise ValueError("Node not in graph")

        if self.weighted is not None:
            self._check_weighted(edge)
        else:
            self.weighted = edge.weighted

        if self.weighted:
            self._edges[edge.src].append((edge.dest, edge.weight))
        else:
            self._edges[edge.src].append(edge.dest)

    def add_undirected_edge(self, edge):
        """Treated as two directed edges."""
        pass

    def children_of(self, node):
        pass

    def has_node(self, node):
        pass

    def __str__(self):
        return ""